# Example of using Model from Hugging Face

Link: https://huggingface.co/nvidia/domain-classifier

Make sure you pip install them to your env before procedding with the code

In [2]:
# Importing the required libraries
import torch
from torch import nn
from transformers import AutoModel, AutoTokenizer, AutoConfig
from huggingface_hub import PyTorchModelHubMixin
import warnings
ignore_warnings = True

In [3]:
# Custom model class that inherits from nn.Module and PyTorchModelHubMixin
class CustomModel(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config):
        super(CustomModel, self).__init__()
        self.model = AutoModel.from_pretrained(config['base_model'])
        self.dropout = nn.Dropout(config['fc_dropout'])
        self.fc = nn.Linear(self.model.config.hidden_size, len(config['id2label']))

    def forward(self, input_ids, attention_mask):
        features = self.model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        dropped = self.dropout(features)
        outputs = self.fc(dropped)
        return torch.softmax(outputs[:, 0, :], dim=1)

In [4]:
# Setup configuration and model
config = AutoConfig.from_pretrained("nvidia/domain-classifier")
tokenizer = AutoTokenizer.from_pretrained("nvidia/domain-classifier")
model = CustomModel.from_pretrained("nvidia/domain-classifier")

In [5]:
# Prepare and process inputs for the model
text_samples = [
    "The content of this magazine is intended for mature audiences and covers topics related to relationships and adult lifestyles.",
    "The latest blockbuster movie has taken the box office by storm, showcasing incredible special effects and a gripping storyline.",
    "The new electric car model boasts an impressive range of 350 miles on a single charge and features cutting-edge autonomous driving technology.",
    "This skincare routine includes natural products that help rejuvenate your skin and maintain a healthy glow.",
    "The novel, set in a dystopian future, explores themes of freedom and control, capturing the reader's imagination with its vivid storytelling.",
    "The manufacturing sector is experiencing a surge in automation, leading to increased efficiency and reduced operational costs.",
    "The latest smartphone features a high-resolution display, powerful processor, and an advanced camera system.",
    "Investors are advised to diversify their portfolios to mitigate risks associated with market volatility.",
    "This recipe for homemade pasta combines fresh ingredients to create a delicious and hearty meal.",
    "The new multiplayer online game has captivated players with its immersive world and challenging quests."
]
inputs = tokenizer(text_samples, return_tensors="pt", padding="longest", truncation=True)
outputs = model(inputs['input_ids'], inputs['attention_mask'])

In [6]:
# Get the predicted classes and convert them to domain labels
predicted_classes = torch.argmax(outputs, dim=1)
predicted_domains = [config.id2label[class_idx.item()] for class_idx in predicted_classes.cpu().numpy()]
print(predicted_domains)

['People_and_Society', 'Arts_and_Entertainment', 'Autos_and_Vehicles', 'Beauty_and_Fitness', 'Books_and_Literature', 'Business_and_Industrial', 'Internet_and_Telecom', 'Finance', 'Food_and_Drink', 'Games']


In [7]:
#printing the input text and the predicted domain just for reference
for text, domain in zip(text_samples, predicted_domains):
    print(f"Text: {text}")
    print(f"Predicted Domain: {domain}")
    print()

Text: The content of this magazine is intended for mature audiences and covers topics related to relationships and adult lifestyles.
Predicted Domain: People_and_Society

Text: The latest blockbuster movie has taken the box office by storm, showcasing incredible special effects and a gripping storyline.
Predicted Domain: Arts_and_Entertainment

Text: The new electric car model boasts an impressive range of 350 miles on a single charge and features cutting-edge autonomous driving technology.
Predicted Domain: Autos_and_Vehicles

Text: This skincare routine includes natural products that help rejuvenate your skin and maintain a healthy glow.
Predicted Domain: Beauty_and_Fitness

Text: The novel, set in a dystopian future, explores themes of freedom and control, capturing the reader's imagination with its vivid storytelling.
Predicted Domain: Books_and_Literature

Text: The manufacturing sector is experiencing a surge in automation, leading to increased efficiency and reduced operational 